# Mohr's Circle using Timoshenko Sign Convension

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider

def plot_stress_and_mohr_circle(angle, sigma_x, sigma_y, tau_xy):
    # Define square vertices
    square = np.array([
        [-0.5, -0.5],
        [0.5, -0.5],
        [0.5, 0.5],
        [-0.5, 0.5],
        [-0.5, -0.5],  # Close the square
    ])

    # Rotation matrix
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                                [np.sin(theta), np.cos(theta)]])

    # Rotate square
    rotated_square = square @ rotation_matrix.T

    # Stress transformation equations with modified shear stress sign convention
    sigma_x_prime = (sigma_x * np.cos(theta)**2 + sigma_y * np.sin(theta)**2 -
                     2 * tau_xy * np.sin(theta) * np.cos(theta))
    sigma_y_prime = (sigma_x * np.sin(theta)**2 + sigma_y * np.cos(theta)**2 +
                     2 * tau_xy * np.sin(theta) * np.cos(theta))
    tau_xy_prime = (-(sigma_y - sigma_x) * np.sin(theta) * np.cos(
        theta) + tau_xy * (np.cos(theta)**2 - np.sin(theta)**2))*-1

    # Right face midpoint
    right_midpoint = np.array([0.5, 0])
    right_midpoint_rotated = right_midpoint @ rotation_matrix.T

    # Top face midpoint
    top_midpoint = np.array([0, 0.5])
    top_midpoint_rotated = top_midpoint @ rotation_matrix.T

    # Normalize vectors for fixed length
    right_normal_vector = np.array([1, 0]) * (1 if sigma_x_prime >= 0 else -1)
    right_shear_vector = np.array([0, 1]) * (1 if tau_xy_prime >= 0 else -1)

    top_normal_vector = np.array([0, 1]) * (1 if sigma_y_prime >= 0 else -1)
    top_shear_vector = np.array([1, 0]) * (1 if tau_xy_prime >= 0 else -1)

    # Rotate vectors
    right_normal_vector_rotated = right_normal_vector @ rotation_matrix.T
    right_shear_vector_rotated = right_shear_vector @ rotation_matrix.T

    top_normal_vector_rotated = top_normal_vector @ rotation_matrix.T
    top_shear_vector_rotated = top_shear_vector @ rotation_matrix.T

    # Normal stress arrows
    right_normal_start = right_midpoint_rotated if sigma_x_prime >= 0 else (right_midpoint_rotated - right_normal_vector_rotated)
    right_normal_end = right_normal_start + right_normal_vector_rotated

    top_normal_start = top_midpoint_rotated if sigma_y_prime >= 0 else (top_midpoint_rotated - top_normal_vector_rotated)
    top_normal_end = top_normal_start + top_normal_vector_rotated

    # Shear stress arrows
    right_shear_center = (right_normal_start + right_normal_end) / 3
    right_shear_start = (right_shear_center - 0.5 * right_shear_vector_rotated)
    right_shear_end = right_shear_start + right_shear_vector_rotated

    top_shear_center = (top_normal_start + top_normal_end) / 3
    top_shear_start = (top_shear_center - 0.5 * top_shear_vector_rotated)
    top_shear_end = top_shear_start + top_shear_vector_rotated

    # Mohr Circle calculations
    center = (sigma_x + sigma_y) / 2
    radius = np.sqrt(((sigma_x - sigma_y) / 2)**2 + tau_xy**2)
    theta_mohr = np.linspace(0, 2 * np.pi, 200)
    mohr_circle_x = center + radius * np.cos(theta_mohr)
    mohr_circle_y = radius * np.sin(theta_mohr)

    # Points on the Mohr Circle
    right_point = (sigma_x_prime, tau_xy_prime)
    top_point = (sigma_y_prime, -tau_xy_prime)

    # Plot the stress element
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))

    # Plot the rotated square
    axes[0].plot(
        rotated_square[:, 0],
        rotated_square[:, 1],
        color="tab:blue",
        ls="-",
        lw=2,
        label="Stress Element",
    )

    # Plot normal stress vectors
    axes[0].quiver(
        right_normal_start[0],
        right_normal_start[1],
        right_normal_vector_rotated[0],
        right_normal_vector_rotated[1],
        angles="xy",
        scale_units="xy",
        scale=1,
        color="r",
        label="Right Normal Stress",
    )
    axes[0].quiver(
        top_normal_start[0],
        top_normal_start[1],
        top_normal_vector_rotated[0],
        top_normal_vector_rotated[1],
        angles="xy",
        scale_units="xy",
        scale=1,
        color="tab:orange",
        label="Top Normal Stress",
    )

    # Plot shear stress vectors
    axes[0].quiver(
        right_shear_start[0],
        right_shear_start[1],
        right_shear_vector_rotated[0],
        right_shear_vector_rotated[1],
        angles="xy",
        scale_units="xy",
        scale=1,
        color="g",
        label="Right Shear Stress",
    )
    axes[0].quiver(
        top_shear_start[0],
        top_shear_start[1],
        top_shear_vector_rotated[0],
        top_shear_vector_rotated[1],
        angles="xy",
        scale_units="xy",
        scale=1,
        color="tab:green",
        label="Top Shear Stress",
    )

    # Annotate stress arrows
    if sigma_x_prime >= 0:
        axes[0].text(
            right_normal_end[0] + 0.1,
            right_normal_end[1] + 0.1,
            f"{sigma_x_prime:.1f}",
            color="k",
            fontsize=12,
        )
    else:
        axes[0].text(
            right_normal_start[0] + 0.1,
            right_normal_start[1] + 0.1,
            f"{sigma_x_prime:.1f}",
            color="k",
            fontsize=12,
        )

    if sigma_y_prime >= 0:
        axes[0].text(
            top_normal_end[0] + 0.1,
            top_normal_end[1] + 0.1,
            f"{sigma_y_prime:.1f}",
            color="k",
            fontsize=12,
        )
    else:
        axes[0].text(
            top_normal_start[0] + 0.1,
            top_normal_start[1] + 0.1,
            f"{sigma_y_prime:.1f}",
            color="k",
            fontsize=12,
        )

    axes[0].text(
        right_shear_end[0] + 0.1,
        right_shear_end[1] + 0.1,
        f"{tau_xy_prime:.1f}",
        color="k",
        fontsize=12,
    )
    axes[0].text(
        top_shear_end[0] + 0.1,
        top_shear_end[1] + 0.1,
        f"{tau_xy_prime:.1f}",
        color="k",
        fontsize=12,
    )

    axes[0].set_xlim(-4, 4)
    axes[0].set_ylim(-4, 4)
    axes[0].axhline(0, color="gray", lw=0.5, linestyle="--")
    axes[0].axvline(0, color="gray", lw=0.5, linestyle="--")
    axes[0].set_title("Stress Element")
    axes[0].set_aspect("equal", adjustable="box")
    axes[0].legend()

    # Plot the Mohr Circle
    axes[1].plot(mohr_circle_x,
                 mohr_circle_y,
                 color="tab:blue",
                 ls="-",
                 label="Mohr Circle")
    axes[1].scatter([sigma_x_prime], [tau_xy_prime],
                    color="r",
                    label="Right Stress Point")
    axes[1].scatter([sigma_y_prime], [-tau_xy_prime],
                    color="tab:orange",
                    label="Top Stress Point")

    # Add the center point to Mohr Circle
    axes[1].scatter([center], [0], color="black", label="_nolegend_")
    axes[1].text(center + 2,
                 2,
                 f"({center:.1f}, 0)",
                 color="black",
                 fontsize=10)

    # Connect stress points (omit label for legend)
    axes[1].plot(
        [sigma_x_prime, sigma_y_prime],
        [tau_xy_prime, -tau_xy_prime],
        color="gray",
        linestyle="--",
    )

    # Annotate stress points
    axes[1].text(
        sigma_x_prime + 2,
        tau_xy_prime + 2,
        f"({sigma_x_prime:.1f}, {tau_xy_prime:.1f})",
        color="r",
        fontsize=10,
    )
    axes[1].text(
        sigma_y_prime + 2,
        -tau_xy_prime + 2,
        f"({sigma_y_prime:.1f}, {-tau_xy_prime:.1f})",
        color="tab:orange",
        fontsize=10,
    )

    # Annotate the rotation angle on the Mohr Circle (2θ) next to the right stress point
    axes[1].text(
        sigma_x_prime + 2,
        tau_xy_prime - 16,
        f"Angle: {2 * angle:.1f}°",
        color="tab:blue",
        fontsize=12,
    )

    axes[1].axhline(0, color="gray", lw=0.5, linestyle="--")
    axes[1].axvline(0, color="gray", lw=0.5, linestyle="--")
    axes[1].set_xlabel("Normal Stress (σ)")
    axes[1].set_ylabel("Shear Stress (τ)")
    axes[1].set_xlim(center - radius - 10, center + radius + 10)
    axes[1].set_ylim(radius + 10, -radius - 10)  # Reverse the y-axis
    axes[1].set_title("Mohr Circle")
    axes[1].legend(loc="upper right")
    axes[1].set_aspect("equal", adjustable="box")

    plt.tight_layout()
    plt.show()

# Interactive sliders for user input
angle_slider = FloatSlider(value=0,
                           min=-180,
                           max=180,
                           step=0.5,
                           description="Angle:")
sigma_x_slider = FloatSlider(value=200,
                             min=-300,
                             max=300,
                             step=10,
                             description="σx:")
sigma_y_slider = FloatSlider(value=-200,
                             min=-300,
                             max=300,
                             step=10,
                             description="σy:")
tau_xy_slider = FloatSlider(value=-300,
                            min=-300,
                            max=300,
                            step=5,
                            description="τxy:")

interact(
    plot_stress_and_mohr_circle,
    angle=angle_slider,
    sigma_x=sigma_x_slider,
    sigma_y=sigma_y_slider,
    tau_xy=tau_xy_slider,
)

interactive(children=(FloatSlider(value=0.0, description='Angle:', max=180.0, min=-180.0, step=0.5), FloatSlid…

<function __main__.plot_stress_and_mohr_circle(angle, sigma_x, sigma_y, tau_xy)>